## Training Bayesian Network

### Get updated precision matrix from data

In [1]:
# Import libraries 
import pandas as pd 
import numpy as np

In [2]:
# Load CO2 and Ethylene gas mixture file
header_names = ["Time", "CO2", "Ethylene", "Sensor1", "Sensor2", "Sensor3", "Sensor4", "Sensor5", "Sensor6", "Sensor7", "Sensor8", "Sensor9", "Sensor10", "Sensor11", "Sensor12", "Sensor13", "Sensor14", "Sensor15", "Sensor16"]
df = pd.read_csv("data/gas-mixture/ethylene_CO.txt", delim_whitespace=True, skiprows=[0], header=None, names=header_names)

# Set time column as index
df = df.set_index("Time")

In [3]:
# Check data frame
df.head()

,CO2,Ethylene,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,Sensor10,Sensor11,Sensor12,Sensor13,Sensor14,Sensor15,Sensor16
Time,,,,,,,,,,,,,,,,,,
0.00,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,55139.95,50669.50,9626.26,9762.62,24544.02,21420.68,7650.61,6928.42
0.01,0.0,0.0,-49.40,-5.53,-42.78,0.49,3.58,-34.55,-9.59,5.37,54395.77,50046.91,9433.20,9591.21,24137.13,20930.33,7498.79,6800.66
0.01,0.0,0.0,-40.04,-16.09,-27.59,0.00,-7.16,-42.14,-12.52,-5.86,53960.02,49299.30,9324.40,9449.81,23628.90,20504.94,7369.67,6697.47
0.03,0.0,0.0,-47.14,-10.57,-32.28,4.40,-11.22,-37.94,-7.16,-1.14,53047.71,48907.00,9170.64,9305.58,23101.66,20101.42,7285.13,6578.52
0.04,0.0,0.0,-33.58,-20.79,-33.25,6.03,3.42,-34.22,-14.46,8.31,52700.28,48330.96,9073.64,9163.47,22689.54,19694.07,7156.74,6468.32


In [4]:
# Transform data frame in numpy matrix
# data = (df.head(1000)).values
data = df.values

In [5]:
# Calculate mean of each column
x_mean = data.mean(0)

In [6]:
# Calculate s
s = data-x_mean
s = (s.T).dot(s)

In [7]:
# Calculate M
M = data.size

In [8]:
# Assuming prior ignorance. v = 0 and beta = 0
beta_update = s
mi_update = x_mean
v_update = M
alpha_update = M-1

In [9]:
# number of random variables
n = 18 

# Calculate Phi_Update
phi_update = (v_update+1)/(v_update*(alpha_update-n+1))
phi_update *= beta_update

In [10]:
# Calculate Precision Update
precision_update = np.linalg.inv(phi_update)

### Calculate Symbolic Precision Matrix

In [11]:
import sympy as sp

In [12]:
# Startup and configuration
n = 18         # 18 random variables
b = [None]*n   # list to initialize bij symbols
v = [None]*n   # list to initialize vi  symbols (variance)

In [13]:
# Create all bij symbols (from b11 to b1818)
for i in range(n):
    symbol_string = ''
    for j in range(i+1):
        symbol_string += 'b' + str(i+1) + '_' + str(j+1) + ' '
    b[i] = sp.symbols(symbol_string)

In [14]:
# Start B matrix using bij symbols
B = sp.zeros(n,n)
B[0, 0] = b[0]       # first symbols is exception not being a list of list
for i in range(1,n):
    for j in range(i+1):
        B[i,j] = b[i][j]

In [15]:
# Create all v, variance symbols (from v1 to v18)
for i in range(n):
    v[i] = sp.symbols(r'\sigma' + str(i+1))

In [16]:
T = sp.zeros(1,1)
T[0,0] = (1/v[0])
for i in range (1, 18):
    # Initialize ti
#    t = (1/v[i])**2
    t = (1/v[i]) # For simplicity variance will be represented without the square
    tt = sp.zeros(1,1)
    tt[0,0] = t
    
    # Initialize  
    TMP = sp.zeros(i+1,i+1)
    
    ## Populate                                                ## row, col
    TMP[0:i,   0:i]   = T + t*sp.Transpose(B[i,0:i])*B[i,0:i]  ## [0 , 0]
    TMP[0:i,   i:i+1] = -sp.Transpose(t*B[i,0:i])              ## [0 , 1]
    TMP[i:i+1, 0:i]   = -t*B[i,0:i]                            ## [1 , 0]
    TMP[i:i+1, i:i+1] = tt                                     ## [1 , 1]
    
    
    T = TMP
# T

### Solve Precision Matrix (get parameters $\sigma_{ij}^*$ and $b_{ij}$)

In [17]:
# Start Equation list
counter = 0
EQ = sp.zeros(171, 1)
for j in reversed(range(0,n)):
    for i in reversed(range(0,j+1)):
        # EQ[(i-1)*n + j] = T[i][j]-precision_update[i][j]        
        EQ[counter] = T[i, j]-precision_update[i, j]
        counter += 1

In [18]:
solution = sp.solve(EQ[0:140], dict=True, manual=True)

In [19]:
solution

[{\sigma10: 86.6964814010731,
  \sigma11: 2229.10172798706,
  \sigma12: 117.981287969481,
  \sigma13: 482.325259658590,
  \sigma14: 53.6899146023380,
  \sigma15: 177.366475648499,
  \sigma16: 9.89085183517062,
  \sigma17: 187.401313076867,
  \sigma18: 146.501421080263,
  \sigma8: 50.1298092341350,
  \sigma9: 1135.68465692175,
  b10_1: 0.0790667559037269,
  b10_2: 0.801785334838480,
  b10_3: 0.00495353996920800,
  b10_4: 0.00246893168261756,
  b10_5: -0.355116621020493,
  b10_6: 0.300302834268022,
  b10_7: 0.127659940382195,
  b10_8: -0.303885774340900,
  b10_9: 1.10257815525530,
  b11_1: 0.555799009653353,
  b11_10: 0.371837084530704,
  b11_2: -2.30762820647062,
  b11_3: 0.636959350421081,
  b11_4: -0.0290435686630750,
  b11_5: -1.69375661270104,
  b11_6: 1.43792049618018,
  b11_7: -3.63749178975224,
  b11_8: 1.10547597951064,
  b11_9: 0.221257415224033,
  b12_1: -0.00925344661016664,
  b12_10: -0.109556891310356,
  b12_11: 0.892668832487929,
  b12_2: -0.0271496313440142,
  b12_3: 0.05

In [20]:
for i in range(len(EQ)):
    EQ[i] = EQ[i].subs(solution[0])

In [21]:
solution1 = sp.solve(EQ[140:171], dict=True, manual=True)

In [22]:
solution1

[{\sigma1: 1436.07355800653,
  \sigma2: 1.75327557172758,
  \sigma3: 19917.2091540482,
  \sigma4: 127660.493593088,
  \sigma5: 68531.9615300373,
  \sigma6: 88.3523544898902,
  \sigma7: 785.983180857272,
  b2_1: -0.00166720447940115,
  b3_1: 3.17862904421874,
  b3_2: 17.4238281746323,
  b4_1: -0.426989760115247,
  b4_2: -30.0205034012173,
  b4_3: 0.229805142897308,
  b5_1: 0.167650573625252,
  b5_2: 166.734351835911,
  b5_3: 0.745561104410979,
  b5_4: -0.0219222611748517,
  b6_1: 0.0926464739279943,
  b6_2: -0.580883989765739,
  b6_3: 0.0820536664203228,
  b6_4: 0.00598026586840613,
  b6_5: 1.05867991271627,
  b7_1: 0.413052966895524,
  b7_2: 5.27769628413856,
  b7_3: 0.170972544776660,
  b7_4: 0.0401333008620466,
  b7_5: -0.119365805723064,
  b7_6: 0.275172514036572,
  b8_1: 0.0902531962021726,
  b8_2: -1.48927890971384,
  b8_3: 0.0248569757798889}]

In [23]:
final_sol = solution[0]

In [24]:
final_sol.update(solution1[0])

In [25]:
final_sol

{\sigma1: 1436.07355800653,
 \sigma10: 86.6964814010731,
 \sigma11: 2229.10172798706,
 \sigma12: 117.981287969481,
 \sigma13: 482.325259658590,
 \sigma14: 53.6899146023380,
 \sigma15: 177.366475648499,
 \sigma16: 9.89085183517062,
 \sigma17: 187.401313076867,
 \sigma18: 146.501421080263,
 \sigma2: 1.75327557172758,
 \sigma3: 19917.2091540482,
 \sigma4: 127660.493593088,
 \sigma5: 68531.9615300373,
 \sigma6: 88.3523544898902,
 \sigma7: 785.983180857272,
 \sigma8: 50.1298092341350,
 \sigma9: 1135.68465692175,
 b10_1: 0.0790667559037269,
 b10_2: 0.801785334838480,
 b10_3: 0.00495353996920800,
 b10_4: 0.00246893168261756,
 b10_5: -0.355116621020493,
 b10_6: 0.300302834268022,
 b10_7: 0.127659940382195,
 b10_8: -0.303885774340900,
 b10_9: 1.10257815525530,
 b11_1: 0.555799009653353,
 b11_10: 0.371837084530704,
 b11_2: -2.30762820647062,
 b11_3: 0.636959350421081,
 b11_4: -0.0290435686630750,
 b11_5: -1.69375661270104,
 b11_6: 1.43792049618018,
 b11_7: -3.63749178975224,
 b11_8: 1.1054759795

In [33]:
import csv

w = csv.writer(open("output.csv", "w"))
for key, val in final_sol.items():
    w.writerow([str(key), val])

In [28]:
type(final_sol)

dict

In [34]:
x_mean

array([1.27620212e+02, 5.32043647e+00, 2.06561664e+03, 5.06218365e+02,
       4.37253865e+03, 4.79924961e+03, 1.89495717e+03, 2.21469248e+03,
       5.08663948e+03, 5.38542821e+03, 1.18624146e+03, 1.21925228e+03,
       4.87107894e+03, 3.92624998e+03, 9.28187700e+02, 1.03577199e+03,
       5.28932324e+03, 4.30968520e+03])